<a href="https://colab.research.google.com/github/Lucius-40/Computer-Vision-Projects/blob/main/CIFAR-10/Object_recognition_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook , i will try a deeper CNN with more units and try a disk saving augmentation approach and dropout and learning rate scheduling to hopefully see improvements in accuracy from the previous notebook

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from PIL import Image
import os
import random
from torchvision.datasets import ImageFolder



# Check GPU availability
torch.cuda.is_available()

False

In [2]:


# Output directory for augmented dataset
OUTPUT_DIR = './augmented_cifar10'
NUM_AUG_PER_IMAGE = 4
IMAGE_SIZE = 32

# CIFAR-10 normalization stats
mean = (0.4914, 0.4822, 0.4465)
std = (0.2470, 0.2435, 0.2616)

# Base spatial transforms
base_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip()
])

# Extra augmentation pool
augmentation_pool = [
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.01, contrast=0.01, saturation=0.01),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=1.0),  # always apply
    transforms.RandomInvert(p=1.0)
]

# Load CIFAR-10 training set (no transform yet)
dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

# Create class-name folders
for class_name in dataset.classes:  # ['airplane', 'automobile', ...]
    os.makedirs(os.path.join(OUTPUT_DIR, class_name), exist_ok=True)

# Loop through dataset and save augmented images
for idx, (image, label) in enumerate(dataset):
    class_name = dataset.classes[label]
    class_dir = os.path.join(OUTPUT_DIR, class_name)

    # Save original image
    image.save(os.path.join(class_dir, f"{idx:05d}_orig.png"))

    # Generate multiple augmented versions
    for aug_idx in range(NUM_AUG_PER_IMAGE):
        img_aug = base_transform(image)  # spatial aug first
        aug = random.choice(augmentation_pool)  # pick one extra aug
        img_aug = aug(img_aug)
        img_aug.save(os.path.join(class_dir, f"{idx:05d}_aug{aug_idx}.png"))

    if idx % 5000 == 0:
        print(f"Processed {idx} images...")

print("Augmentation complete! Images saved in class-name folders.")




100%|██████████| 170M/170M [00:03<00:00, 47.9MB/s]


Processed 0 images...
Processed 5000 images...
Processed 10000 images...
Processed 15000 images...
Processed 20000 images...
Processed 25000 images...
Processed 30000 images...
Processed 35000 images...
Processed 40000 images...
Processed 45000 images...
Augmentation complete! Images saved in class-name folders.


In [7]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


train_dataset = ImageFolder(root=OUTPUT_DIR, transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

print(f"Total images: {len(train_dataset)}")
print(f"Classes: {train_dataset.classes}")  # Matches CIFAR-10 exactly


test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_train
)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

print(f"Test classes: {test_dataset.classes}")  # Same order as train_dataset.classes

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)


Total images: 250000
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Test classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [8]:
print("total Images ", len(test_dataset))
print(test_dataset.classes)

total Images  10000
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


**New Model **

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)


        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)

        self.dense1 = nn.Linear(4 * 4* 128, 150)
        self.batchnorm4 = nn.BatchNorm1d(150)

        self.dense2 = nn.Linear(150, 120 )
        self.batchnorm5 = nn.BatchNorm1d(120)

        self.output = nn.Linear(120,10)

        self.pool = nn.MaxPool2d(2,2)
        self.conv_drop = nn.Dropout2d(p=0.1)
        self.dense_drop = nn.Dropout(p=0.2)

    def forward(self, X):
        X = self.pool(torch.relu(self.batchnorm1(self.conv1(X))))
        X= self.conv_drop(X)

        X = self.pool(torch.relu(self.batchnorm2(self.conv2(X))))
        X=self.conv_drop(X)

        X = self.pool(torch.relu(self.batchnorm3(self.conv3(X))))
        X=self.conv_drop(X)

        X = X.view(X.size(0),-1 )
        X = torch.relu(self.batchnorm4(self.dense1(X)))
        X=self.dense_drop(X)

        X = torch.relu(self.batchnorm5(self.dense2(X)))
        X=self.dense_drop(X)

        X = self.output(X)
        return X

In [10]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.6,
    weight_decay=1e-4
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=70, eta_min=1e-5)


The training loop :

In [ ]:
'''
for epoch in range(70):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%")
    '''
for epoch in range(70):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_acc = 100 * correct_test / total_test


    scheduler.step()

    print(f"Epoch [{epoch+1}/70] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% "
          f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

